In [30]:
import pandas as pd
import random
import os 
import json
import traceback

from datetime import datetime

In [39]:
class ValidadorCampos:
    def __init__(self, ruta_excel, ruta_json, ruta_salida):
        self.reglas = {}
        self.ruta_excel = ruta_excel
        self.ruta_json = ruta_json
        self.ruta_salida = ruta_salida        
        self.lista_fields_transformados = self.leer_json()        

    def cargar_reglas_desde_excel(self):
        try:
            df = pd.read_excel(self.ruta_excel)
            df.columns = df.columns.str.upper()
            
            for columna in df.select_dtypes(include='object').columns:
                df[columna] = df[columna].str.upper()
            
            columnas_requeridas = ["BB", "CAMPO DESTINO", "TIPO", "LONGITUD"]
            for columna in columnas_requeridas:
                if columna not in df.columns:
                    raise ValueError(f"La columna '{columna}' no existe en el archivo Excel")
            
            for _, fila in df.iterrows():
                nombre_campo = fila["BB"]
                codigo = fila["CAMPO DESTINO"]
                tipo = fila["TIPO"]
                
                longitud = fila["LONGITUD"]
                if pd.isna(longitud):
                    longitud = 0
                else:
                    longitud = int(longitud)

                self.reglas[nombre_campo] = {
                    'codigo': codigo,
                    'tipo': tipo,
                    'longitud': longitud
                }
            
            print(f"Se cargaron {len(self.reglas)} reglas desde el archivo Excel")            
        except Exception as e:
            print(f"Error al cargar las reglas desde Excel: {e}")
            raise

    def leer_json(self):
        try:
            with open(self.ruta_json, 'r', encoding='utf-8') as f:
                datos = json.load(f)
                transactions = datos.get("body", {}).get("transactions", [])
                lista_fields_transformados = []
                
                for transaccion in transactions:
                    if "fields" in transaccion:
                        nuevo_fields = {}

                        for campo, valor in transaccion["fields"].items():
                            regla = self.reglas.get(campo, None)
                            
                            if not regla:
                                nuevo_fields[campo] = valor
                                continue  
                            
                            codigo = regla.get("codigo", campo)
                            tipo = regla.get("tipo", "CARACTER")  
                            longitud = regla.get("longitud", 0)

                            valor = str(valor).strip()
                            
                            if tipo in ["FECINT", "DATETIME"]:
                                try:
                                    fecha = datetime.strptime(valor, "%Y-%m-%d")
                                    valor = fecha.strftime("%Y%m%d")
                                except ValueError:
                                    print(f"⚠️ Advertencia: Formato de fecha inválido en {campo}: {valor}")
                            
                            if tipo == "NUMERICO":
                                valor = valor.ljust(longitud, '0')[:longitud]  
                            else:  
                                valor = valor.rjust(longitud, '0')[:longitud]  

                            nuevo_fields[codigo] = valor
                        
                        lista_fields_transformados.append(nuevo_fields)
                
                return lista_fields_transformados
        except Exception as e:
            print(f"Error al leer o transformar el JSON:\n{traceback.format_exc()}")

    def generar_archivo_txt(self):
        """Genera un archivo .txt con los datos transformados en formato tabla"""
        try:
            if not self.lista_fields_transformados:
                print("No hay datos transformados para exportar.")
                return            
            
            columnas = list(self.lista_fields_transformados[0].keys())
            
            ancho_columnas = {col: max(len(col), 15) for col in columnas}  
            for fila in self.lista_fields_transformados:
                for col, val in fila.items():
                    ancho_columnas[col] = max(ancho_columnas[col], len(str(val)))
            
            encabezado = " ".join(col.ljust(ancho_columnas[col]) for col in columnas)
            
            filas = []
            for fila in self.lista_fields_transformados:
                valores = [str(fila[col]).ljust(ancho_columnas[col]) for col in columnas]
                filas.append(" ".join(valores))
            
            with open(self.ruta_salida, "w", encoding="utf-8") as f:
                f.write(encabezado + "\n")
                f.write("\n".join(filas) + "\n")

            print(f"Archivo guardado en {self.ruta_salida}")

        except Exception as e:
            print(f"Error al generar el archivo TXT:\n{traceback.format_exc()}")

In [40]:
random_int = random.randint(1, 10000)

ruta_excel = "./data/Reglas OPTIMA.xlsx"
ruta_json = "./data/json_a_homologar.json"
ruta_salida = f"./outputs/output_{random_int}.txt"
if not os.path.exists(ruta_salida):  
    random_int = random.randint(1, 10000)
    ruta_salida = f"./outputs/output_{random_int}.txt"

validador = ValidadorCampos(ruta_excel, ruta_json, ruta_salida)
validador.cargar_reglas_desde_excel()
validador.generar_archivo_txt()

Se cargaron 29 reglas desde el archivo Excel
Archivo guardado en ./outputs/output_1175.txt
